In [1]:
import pandas as pd

In [2]:
df= pd.read_csv('Workers__Compensation_Injury___Exposure_Filtered_View.csv')
df

C:\Users\Ismail\AppData\Local\Temp\ipykernel_25700\2793983325.py:1: DtypeWarning: Columns (12,13,14,15,16,17,18,19,20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  df= pd.read_csv('Workers__Compensation_Injury___Exposure_Filtered_View.csv')


,index,Assembly Date,Accident Date,Age at Injury,Gender,Controverted Date,WCIO Part Of Body Code,WCIO Part Of Body Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,...,OIICS Injury Source Description,OIICS Event Exposure Code,OIICS Event Exposure Description,OIICS Secondary Source Code,OIICS Secondary Source Description,Highest Process,Hearing Count,Claim Injury Type,Claim Type,Carrier Type
0,0,01/06/2000,12/08/1998,49.0,M,NaN,NaN,NaN,NaN,NaN,...,Containers--nonpressurized,22*,Overexertion,11*,Containers--nonpressurized,Hearing - Judge,1.0,PPD SCH LOSS,Workers Compensation,SIF
1,1,01/10/2000,10/29/1999,30.0,M,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Administrative Determination,0.0,TEMPORARY,Workers Compensation,PRIVATE
2,2,01/13/2000,12/27/1999,46.0,F,NaN,NaN,NaN,NaN,NaN,...,Furniture,13*,Fall on same level,23*,Furniture,Hearing - Judge,3.0,PPD SCH LOSS,Workers Compensation,PRIVATE
3,3,01/10/2000,11/28/1999,59.0,F,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Conciliation - Desk Decision,0.0,TEMPORARY,Workers Compensation,SIF
4,4,01/03/2000,11/01/1999,72.0,F,NaN,NaN,NaN,NaN,NaN,...,Person--other than injured or ill worker,61*,Assaults and violent acts by person(s),57*,Person--other than injured or ill worker,Hearing - Judge,2.0,PPD SCH LOSS,Workers Compensation,SELF PUBLIC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2729789,2729789,01/28/2016,12/29/2014,NaN,M,NaN,10.0,MULTIPLE HEAD INJURY,49.0,SPRAIN OR TEAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-COMP,Workers Compensation,NaN
2729790,2729790,01/29/2016,01/08/2016,NaN,M,NaN,53.0,KNEE,52.0,STRAIN OR TEAR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-COMP,Workers Compensation,NaN
2729791,2729791,01/29/2016,11/10/2015,NaN,M,NaN,31.0,UPPER ARM,10.0,CONTUSION,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON-COMP,Workers Compensation,NaN
2729792,2729792,02/10/2015,01/22/2015,NaN,F,NaN,53.0,KNEE,10.0,CONTUSION,...,NaN,NaN,NaN,NaN,NaN,No Resolutions,NaN,NON-COMP,Workers Compensation,NaN


In [3]:
df.isnull().sum()

index                                       0
Assembly Date                               0
Accident Date                           16097
Age at Injury                          748097
Gender                                      0
Controverted Date                     2446343
WCIO Part Of Body Code                2029263
WCIO Part Of Body Description         2029263
WCIO Nature of Injury Code            2031173
WCIO Nature of Injury Description     2031173
WCIO Cause of Injury Code             2030917
WCIO Cause of Injury Description      2030917
OIICS Part Of Body Code               1650920
OIICS Part Of Body Description        1650920
OIICS Nature of Injury Code           1650920
OIICS Nature of Injury Description    1650920
OIICS Injury Source Code              1650932
OIICS Injury Source Description       1650932
OIICS Event Exposure Code             1650929
OIICS Event Exposure Description      1650929
OIICS Secondary Source Code           1650965
OIICS Secondary Source Description

In [4]:
df.shape

(2729794, 27)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2729794 entries, 0 to 2729793
Data columns (total 27 columns):
 #   Column                              Dtype  
---  ------                              -----  
 0   index                               int64  
 1   Assembly Date                       object 
 2   Accident Date                       object 
 3   Age at Injury                       float64
 4   Gender                              object 
 5   Controverted Date                   object 
 6   WCIO Part Of Body Code              float64
 7   WCIO Part Of Body Description       object 
 8   WCIO Nature of Injury Code          float64
 9   WCIO Nature of Injury Description   object 
 10  WCIO Cause of Injury Code           float64
 11  WCIO Cause of Injury Description    object 
 12  OIICS Part Of Body Code             object 
 13  OIICS Part Of Body Description      object 
 14  OIICS Nature of Injury Code         object 
 15  OIICS Nature of Injury Description  object 
 16  

In [7]:
import os
import sys
import gc
import numpy as np
from sklearn.impute import SimpleImputer

In [8]:
# -----------------------------
# CONFIG
# -----------------------------
INPUT_PATH = "Workers__Compensation_Injury___Exposure_Filtered_View.csv"         
OUTPUT_PATH = "claims_cleaned.csv"
MISSING_COL_THRESHOLD_DROP = 0.80      # drop columns with >80% missing
CATEGORICAL_FILL_MODE = True
NUMERIC_IMPUTE_STRATEGY = "median"     # median recommended for Age
RANDOM_SEED = 42

In [9]:
# -----------------------------
# HELPERS
# -----------------------------
def mem_report(df, name="df"):
    """Print memory usage of dataframe."""
    print(f"{name} memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

def missing_summary(df):
    """Return dataframe with missing counts and percentages sorted descending."""
    miss = df.isnull().sum().rename("missing_count").to_frame()
    miss["missing_pct"] = miss["missing_count"] / len(df)
    return miss.sort_values("missing_pct", ascending=False)

def optimize_dtypes(df, cat_cols=None, int_cols=None, float_cols=None):
    """
    Convert obvious columns to smaller dtypes to save memory:
    - object -> category for categorical cols (if provided or heuristics).
    - downcast numeric types where safe.
    """
    # heuristics: convert object with low cardinality to category
    for col in df.select_dtypes(include=["object"]).columns:
        # skip if obviously long free-text (heuristic: avg length > 50)
        avg_len = df[col].dropna().map(lambda x: len(str(x))).mean() if df[col].dropna().shape[0] > 0 else 0
        nunique = df[col].nunique(dropna=True)
        if cat_cols and col in cat_cols:
            df[col] = df[col].astype("category")
        elif nunique / len(df) < 0.5 and avg_len < 50:
            df[col] = df[col].astype("category")
    # downcast ints and floats
    df = df.copy()
    for col in df.select_dtypes(include=["int64"]).columns:
        df[col] = pd.to_numeric(df[col], downcast="unsigned")
    for col in df.select_dtypes(include=["float64"]).columns:
        df[col] = pd.to_numeric(df[col], downcast="float")
    return df

In [10]:
# -----------------------------
# LOAD (with safe dtype hints)
# -----------------------------
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"Input file not found at {INPUT_PATH}. Please set INPUT_PATH to your CSV.")

# Try reading with parse_dates for known date columns to save later conversions
parse_dates = ["Accident Date", "Assembly Date", "Controverted Date"]
# If parse_dates not present in file, pandas will ignore non-existing names.
print("Loading dataset (this may take a while for large files)...")
df = pd.read_csv(INPUT_PATH, parse_dates=parse_dates, low_memory=False)
print("Loaded data shape:", df.shape)
mem_report(df, "raw_df")

Loading dataset (this may take a while for large files)...
Loaded data shape: (2729794, 27)
raw_df memory usage: 2439.35 MB


In [11]:
# -----------------------------
# INITIAL EXPLORATION
# -----------------------------
print("\n--- Missing value summary (top) ---")
ms = missing_summary(df)
print(ms.head(30))   # show top 30 missing columns
print("\nTotal rows:", len(df))


--- Missing value summary (top) ---
                                    missing_count  missing_pct
Controverted Date                         2446343     0.896164
WCIO Nature of Injury Description         2031173     0.744076
WCIO Nature of Injury Code                2031173     0.744076
WCIO Cause of Injury Description          2030917     0.743982
WCIO Cause of Injury Code                 2030917     0.743982
WCIO Part Of Body Code                    2029263     0.743376
WCIO Part Of Body Description             2029263     0.743376
OIICS Secondary Source Description        1650965     0.604795
OIICS Secondary Source Code               1650965     0.604795
OIICS Injury Source Description           1650932     0.604783
OIICS Injury Source Code                  1650932     0.604783
OIICS Event Exposure Code                 1650929     0.604782
OIICS Event Exposure Description          1650929     0.604782
OIICS Part Of Body Code                   1650920     0.604778
OIICS Nature of In

In [12]:
# -----------------------------
# DROP COLUMNS WITH VERY HIGH MISSING RATE
# -----------------------------
# These are costly to impute and often not useful if >80% missing.
drop_cols = ms[ms["missing_pct"] > MISSING_COL_THRESHOLD_DROP].index.tolist()
if drop_cols:
    print(f"\nDropping columns with >{int(MISSING_COL_THRESHOLD_DROP*100)}% missing: {drop_cols}")
    df.drop(columns=drop_cols, inplace=True)
else:
    print("\nNo columns exceed missing threshold for dropping.")


Dropping columns with >80% missing: ['Controverted Date']


In [13]:

# Recompute missing summary
ms = missing_summary(df)
print("\nMissing after dropping high-missing cols (top 30):")
print(ms.head(30))


Missing after dropping high-missing cols (top 30):
                                    missing_count  missing_pct
WCIO Nature of Injury Code                2031173     0.744076
WCIO Nature of Injury Description         2031173     0.744076
WCIO Cause of Injury Code                 2030917     0.743982
WCIO Cause of Injury Description          2030917     0.743982
WCIO Part Of Body Code                    2029263     0.743376
WCIO Part Of Body Description             2029263     0.743376
OIICS Secondary Source Description        1650965     0.604795
OIICS Secondary Source Code               1650965     0.604795
OIICS Injury Source Description           1650932     0.604783
OIICS Injury Source Code                  1650932     0.604783
OIICS Event Exposure Code                 1650929     0.604782
OIICS Event Exposure Description          1650929     0.604782
OIICS Part Of Body Code                   1650920     0.604778
OIICS Part Of Body Description            1650920     0.604778
OII

In [14]:
# -----------------------------
# ESSENTIAL DATE: Accident Date
# -----------------------------
# Accident Date is essential for any time series / trend analysis.
accident_missing = df["Accident Date"].isnull().sum() if "Accident Date" in df.columns else 0
print(f"\nAccident Date missing count: {accident_missing} ({accident_missing/len(df):.2%})")

# If small proportion of missing -> drop rows with missing Accident Date
ACCDATE_DROP_PCT = 0.01  # threshold: if missing <=1% we drop those rows
if "Accident Date" in df.columns and (accident_missing / len(df)) <= ACCDATE_DROP_PCT:
    print("Dropping rows with missing Accident Date (small proportion).")
    df = df[~df["Accident Date"].isnull()].copy()
else:
    print("Accident Date has more missing than threshold or is absent; consider careful imputation or using Assembly Date.")

print("Shape after Accident Date handling:", df.shape)
mem_report(df, "post_accident_handling_df")


Accident Date missing count: 16097 (0.59%)
Dropping rows with missing Accident Date (small proportion).
Shape after Accident Date handling: (2713697, 26)
post_accident_handling_df memory usage: 2426.90 MB


In [15]:
# -----------------------------
# REMOVE DUPLICATES and basic integrity checks
# -----------------------------
# If an index column exists that's unique use it; otherwise drop exact duplicates.
if "index" in df.columns:
    # if index column has unique values we can set it as index
    if df["index"].is_unique:
        df.set_index("index", inplace=True)
        print("Set 'index' column as DataFrame index.")
    else:
        # still safe to drop exact duplicate rows
        dup_count = df.duplicated().sum()
        if dup_count > 0:
            print(f"Dropping {dup_count} exact duplicate rows.")
            df.drop_duplicates(inplace=True)
else:
    dup_count = df.duplicated().sum()
    if dup_count > 0:
        print(f"Dropping {dup_count} exact duplicate rows.")
        df.drop_duplicates(inplace=True)

print("Shape after duplicate removal:", df.shape)

Set 'index' column as DataFrame index.
Shape after duplicate removal: (2713697, 25)


In [16]:
# -----------------------------
# AGE: Imputation & flag
# -----------------------------
if "Age at Injury" in df.columns:
    age_missing_count = df["Age at Injury"].isnull().sum()
    print(f"\nAge missing before imputation: {age_missing_count} ({age_missing_count/len(df):.2%})")
    # create flag for missing age - useful for modeling
    df["Age_missing_flag"] = df["Age at Injury"].isnull().astype("uint8")
    # Impute with median (preferred over mean because of outliers)
    if df["Age at Injury"].notnull().sum() > 0:
        median_age = int(df["Age at Injury"].median(skipna=True))
        print(f"Imputing missing Age at Injury with median = {median_age}")
        df["Age at Injury"] = df["Age at Injury"].fillna(median_age).astype("int16")
    else:
        print("No non-null Age values available -> leave as is and keep flag.")


Age missing before imputation: 732433 (26.99%)
Imputing missing Age at Injury with median = 43


In [17]:
# -----------------------------
# CATEGORICAL FIELDS: fill missing with mode or placeholder
# -----------------------------
# Choose categorical columns we want to preserve for analysis
prefer_cats = [
    "Gender", "Claim Injury Type", "Claim Type", "Carrier Type", "Highest Process",
    "OIICS Part Of Body Description", "OIICS Nature of Injury Description",
    "OIICS Injury Source Description", "OIICS Event Exposure Description",
    "OIICS Secondary Source Description",
    "WCIO Part Of Body Description", "WCIO Nature of Injury Description",
    "WCIO Cause of Injury Description"
]

# Keep only columns that exist in df
prefer_cats = [c for c in prefer_cats if c in df.columns]

for col in prefer_cats:
    missing_before = df[col].isnull().sum()
    if missing_before > 0:
        # If mode exists, use it, otherwise fill with "Unknown"
        try:
            mode_val = df[col].mode(dropna=True)[0]
            df[col] = df[col].fillna(mode_val)
            print(f"Filled missing categorical '{col}' with mode: '{mode_val}' ({missing_before} fills).")
        except IndexError:
            df[col] = df[col].fillna("Unknown")
            print(f"No mode for '{col}', filled with 'Unknown' ({missing_before} fills).")
    # convert to category dtype (saves memory)
    df[col] = df[col].astype("category")

Filled missing categorical 'Carrier Type' with mode: 'PRIVATE' (578783 fills).
Filled missing categorical 'Highest Process' with mode: 'Hearing - Judge' (311030 fills).
Filled missing categorical 'OIICS Part Of Body Description' with mode: 'Back, including spine, spinal cord' (1636365 fills).
Filled missing categorical 'OIICS Nature of Injury Description' with mode: 'Traumatic injuries to muscles, tendons, joints, etc.' (1636365 fills).
Filled missing categorical 'OIICS Injury Source Description' with mode: 'Floors, walkways, ground surfaces' (1636377 fills).
Filled missing categorical 'OIICS Event Exposure Description' with mode: 'Overexertion' (1636374 fills).
Filled missing categorical 'OIICS Secondary Source Description' with mode: 'Person--other than injured or ill worker' (1636410 fills).
Filled missing categorical 'WCIO Part Of Body Description' with mode: 'LOWER BACK AREA' (2013206 fills).
Filled missing categorical 'WCIO Nature of Injury Description' with mode: 'STRAIN OR TEAR

In [18]:
# -----------------------------
# NUMERIC fields: ensure dtype and handle missing
# -----------------------------
# Example numeric columns that may exist: Hearing Count
if "Hearing Count" in df.columns:
    # replace obvious invalid negative counts with NaN and then impute with 0 or median
    df["Hearing Count"] = pd.to_numeric(df["Hearing Count"], errors="coerce")
    neg_count = (df["Hearing Count"] < 0).sum()
    if neg_count > 0:
        print(f"Found {neg_count} negative Hearing Count values, setting them to NaN.")
        df.loc[df["Hearing Count"] < 0, "Hearing Count"] = np.nan
    # fill missing with 0 if it makes sense (no hearings), else median
    hearing_missing = df["Hearing Count"].isnull().sum()
    if hearing_missing / len(df) < 0.2:
        # plausible to assume no hearing => 0
        df["Hearing Count"] = df["Hearing Count"].fillna(0).astype("uint8")
        print(f"Filled Hearing Count missing with 0 (count: {hearing_missing}).")
    else:
        med = int(df["Hearing Count"].median(skipna=True))
        df["Hearing Count"] = df["Hearing Count"].fillna(med).astype("int16")
        print(f"Large missing in Hearing Count; imputed with median {med}.")

Large missing in Hearing Count; imputed with median 1.


In [19]:
# -----------------------------
# DATES: ensure correct dtype & create time features
# -----------------------------
date_cols = [c for c in ["Accident Date", "Assembly Date", "Controverted Date"] if c in df.columns]
for dc in date_cols:
    # coerce to datetime safely
    df[dc] = pd.to_datetime(df[dc], errors="coerce")
    print(f"Converted {dc} to datetime. NaT count: {df[dc].isnull().sum()}")

# Create Year/Month/Weekday for trend analysis from Accident Date
if "Accident Date" in df.columns:
    df["Accident_Year"] = df["Accident Date"].dt.year.astype("Int32")
    df["Accident_Month"] = df["Accident Date"].dt.month.astype("Int8")
    df["Accident_MonthName"] = df["Accident Date"].dt.month_name().astype("category")
    df["Accident_Weekday"] = df["Accident Date"].dt.day_name().astype("category")
    print("Created Accident_Year / Accident_Month / Accident_MonthName / Accident_Weekday features.")

Converted Accident Date to datetime. NaT count: 0
Converted Assembly Date to datetime. NaT count: 0
Created Accident_Year / Accident_Month / Accident_MonthName / Accident_Weekday features.


In [20]:
# -----------------------------
# CLEAN TEXT: strip whitespace & uniform case for categorical descriptions
# -----------------------------
for col in df.select_dtypes(include=["category", "object"]).columns:
    # skip very long text columns by heuristic
    # apply strip and unify case for categorical-ish columns
    try:
        # convert to string then strip
        df[col] = df[col].astype(str).str.strip()
    except Exception:
        pass

In [21]:
# -----------------------------
# OPTIONAL: reduce cardinality on high-card columns (if needed)
# -----------------------------
# Example: if some description column has too many unique values but mostly "Other", keep top K
def reduce_cardinality(series, top_k=50, other_label="Other"):
    top = series.value_counts().nlargest(top_k).index
    return series.where(series.isin(top), other_label)

# If any description column huge cardinality, reduce it (configurable)
cols_to_reduce = [c for c in ["OIICS Injury Source Description", "OIICS Event Exposure Description",
                              "OIICS Part Of Body Description"] if c in df.columns]

for c in cols_to_reduce:
    nunique = df[c].nunique(dropna=True)
    if nunique > 200:
        print(f"Reducing cardinality of {c} from {nunique} to top 100 + 'Other'.")
        df[c] = reduce_cardinality(df[c], top_k=100).astype("category")

In [22]:
# -----------------------------
# TYPE OPTIMIZATION
# -----------------------------
df = optimize_dtypes(df)
mem_report(df, "optimized_df")

optimized_df memory usage: 176.06 MB


In [27]:
# -----------------------------
# FINAL CHECKS & SAVE
# -----------------------------
print("\nFinal missing summary (top 30):")
print(missing_summary(df).head(30))

# Clean up and exit
gc.collect()
print("\nData preprocessing completed successfully.")


Final missing summary (top 30):
                                    missing_count  missing_pct
WCIO Nature of Injury Code                2015116     0.742572
WCIO Cause of Injury Code                 2014861     0.742478
WCIO Part Of Body Code                    2013206     0.741868
Assembly Date                                   0     0.000000
OIICS Event Exposure Description                0     0.000000
Accident_MonthName                              0     0.000000
Accident_Month                                  0     0.000000
Accident_Year                                   0     0.000000
Age_missing_flag                                0     0.000000
Carrier Type                                    0     0.000000
Claim Type                                      0     0.000000
Claim Injury Type                               0     0.000000
Hearing Count                                   0     0.000000
Highest Process                                 0     0.000000
OIICS Secondary Source

In [26]:
# Good practice: put your cleaned data in "data/" folder
OUTPUT_PATH = "data/claims_cleaned.csv"
os.makedirs(os.path.dirname(OUTPUT_PATH), exist_ok=True)

df.to_csv(OUTPUT_PATH, index=False)
print(f"Saved cleaned dataset to {OUTPUT_PATH}")

Saved cleaned dataset to data/claims_cleaned.csv
